In [1]:
import pandas as pd


def decide_phase1(row):
    if row.get('No Turn On Red', None) == 'Yes':
        if  (row['우회전신호등'] == 'red' or row['우회전신호등'] == 'yellow') or (row['우회전신호등'] == 'No' and (row['신호등1'] == 'red' or row['신호등1'] == 'yellow' )):
            return 'stop'
        elif row['보행자1'] == 'Yes':
            return 'wait for ped'
        return 'slowly go'

    elif row.get('Right On Red After Stop', None) == 'Yes':

        if  (row['우회전신호등'] == 'red' or row['우회전신호등'] == 'yellow') or (row['우회전신호등'] == 'No' and (row['신호등1'] == 'red' or row['신호등1']=='yellow')):
            return 'stop and go'
        elif row['보행자1'] == 'Yes':
            return 'wait for ped'
        return 'slowly go'

    elif row['우회전신호등'] != 'No':
        if (row['우회전신호등'] == 'red' or row['우회전신호등'] == 'yellow') or (row['우회전신호등'] == 'No' and (row['신호등1'] == 'red' or row['신호등1']=='yellow')):
            return 'stop'
        elif row['보행자1'] == 'Yes':
            return 'wait for ped'
        return 'slowly go'

    elif row['신호등1'] == 'red':
        return 'stop'


    return 'slowly go'


def decide_phase2(row, phase1):
    if phase1 in ['stop']:
        return '-'

    if row['차량'] == 'Yes' or row['보행자2'] == 'Yes':
        return 'wait for ped or car'

    return 'right turn'


def newyork_action(row):
    phase1 = decide_phase1(row)
    phase2 = decide_phase2(row, phase1)
    return pd.Series([phase1, phase2], index=['phase1', 'phase2'])


def korea_action(row):
    phase1, phase2 ="slowly go", "right turn"

    no_right_turn_on_red_exists = row.get('No Turn On Red', None) == 'Yes'

    if row["신호등1"] == "red":
        phase1 = "stop and go"
    if row["신호등1"] == "yellow" and (not no_right_turn_on_red_exists or row["No Turn On Red"] == "No"):
        phase1 = "stop and go"
    if no_right_turn_on_red_exists and row["No Turn On Red"] == "Yes" and (row["신호등1"] == "red" or row["신호등1"]== 'yellow' ):
        phase1= "stop"

    if row["우회전신호등"]=="green":
        phase1="slowly go"
    elif row["우회전신호등"] == "red" or row['우회전신호등'] == 'yellow':
        phase1= "stop"

    if row["보행자1"] == "Yes" and phase1!="stop":
        phase1 = "wait for ped"

    if phase1=="stop":
        phase2="-"
    else:
        if row["보행자2"] == "Yes" or row["차량"] == "Yes":
            phase2 = "wait for ped or car"





    return pd.Series([phase1, phase2], index=['phase1', 'phase2'])

In [4]:
import pandas as pd
import itertools


A = pd.read_excel('New York.xlsx')
B = pd.read_excel('Korea.xlsx')

common_component = list(set(A.columns) & set(B.columns))
common_component

A_only_columns = list(set(A.columns) - set(B.columns))
B_only_columns = list(set(B.columns) - set(A.columns))

In [8]:
result_df = pd.DataFrame(columns=B.columns.tolist() + ['phase1', 'phase2'])


#A에도 있는 행인지 확인하기 위한 함수
def check_row_exists_in(row_B, compare):
    temp_df = pd.DataFrame([row_B])
    for col in compare:
        temp_df[col] = 'No'
    merged = A.merge(temp_df, how='inner')
    return not merged.empty


#common component state가 동일하고
#A_only component, B_only component가 No인 case 중
#A와 B의 action이 다른 case
same_state = 0
simillar_state = 0

for _, row in B.iterrows():
    if check_row_exists_in(row, A_only_columns):
      action_korea = korea_action(row)
      korea_row = pd.concat([row, action_korea])
      #korea_result_df = pd.concat([korea_result_df, korea_row.to_frame().T], ignore_index=True)
      action_newyork = newyork_action(row)
      if not action_korea.equals(action_newyork):
        new_row = pd.concat([row, action_korea])
        same_state +=1
        result_df = pd.concat([result_df, new_row.to_frame().T], ignore_index=True)

#B_only componenet가 No가 아닌 모든 case 중
#state가 동일하고 B_only component가 동일한 case와 action이 다른 case

result = pd.DataFrame(columns=B.columns.tolist() + ['phase1', 'phase2'])

for i in range(len(B_only_columns)):
  C=B.copy()
  C=C[C[B_only_columns[i]]!="No"]
  for _, row in C.iterrows():
    B_row=row.copy()
    B_row[B_only_columns[i]]="No"
    action1=korea_action(row)
    action2=korea_action(B_row)
    if not action1.equals(action2):
      new_row=pd.concat([row, action1])

      if not result.isin([new_row]).all(axis=1).any():
        simillar_state += 1
        result = pd.concat([result, new_row.to_frame().T], ignore_index=True)

print(same_state)
print(simillar_state)
answer=pd.concat([result_df, result]).drop_duplicates()
#korea_all =pd.concat([korea_result_df, result]).drop_duplicates()
answer.to_csv('Result_Korea.csv', index=False)

10
0
